# Problem B1: First autocorrelation inequality
For any function $f:\mathbb{R} \rightarrow \mathbb{R}$, define the *autoconvolution* of $f$, written $f*f$, as
$$f*f (t) := \int_\mathbb{R} f(t-x) f(x)\ dx.$$

Let $C_1$ denote the largest constant for which one has
\begin{equation}
 \max_{-1/2 \leq t \leq 1/2} f*f(t) \geq C_1 \left(\int_{-1/4}^{1/4} f(x)\ dx\right)^2
\end{equation}
for all non-negative $f: \mathbb{R} \rightarrow \mathbb{R}$.  This problem arises in additive combinatorics, relating to the size of Sidon sets.  It is currently known that
$$ 1.28 \leq C_1 \leq 1.5098$$
with the lower bound proven by [Cloninger and Steinerberger (2017)](https://www.ams.org/journals/proc/2017-145-08/S0002-9939-2017-13690-9/S0002-9939-2017-13690-9.pdf) and the upper bound achieved by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001) via a step function construction. AlphaEvolve found a step function with 600 equally-spaced intervals on $[-1/4,1/4]$ that gives a better upper bound of $C_1 \leq 1.5053$.

## Compute the step function, the integral, and the autoconvolution

For step func. 
$$f(x) = \sum_{i=0}^{P-1} h_i \mathbf{1}_{[x_i, x_{i+1})}(x)$$  
where
$$ x_i = -\frac{1}{4} + \frac{i}{2P}, \text{ and heights } h_i \ge 0:$$
$$
\begin{aligned}
% Integral
\text{Integral:}\quad \int_{-1/4}^{1/4} f(x) \,dx &= \frac{1}{2P} \sum_{i=0}^{P-1} h_i
\\[2ex]
% Autoconvolution
\text{Autoconv. knots:}\quad (f*f)(t_m) &= 
\begin{cases} 
0 & \text{if } m=0 \text{ or } m=2P \\
\frac{1}{2P} \sum_{k=\max(0, m-P)}^{\min(P-1, m-1)} h_k h_{m-1-k} & \text{if } 1 \le m \le 2P-1
\end{cases}
\\[0.5ex]
&\quad \text{where } t_m = -\frac{1}{2} + \frac{m}{2P} \quad (m=0, \dots, 2P).
\end{aligned}
$$

## Plot the step function and the autoconvolution

In [ ]:
# script assumes 'compute_integral_of_step_function' and 'compute_autoconvolution_values'
# (from your previous code block) are already defined and executable in the notebook.
from utils import *

# --- main script logic ---
P_val = 600  # number of pieces for the step function
f_interval = (-0.25, 0.25) # interval for f(x)

f_x_min, f_x_max = f_interval
f_delta_x = (f_x_max - f_x_min) / P_val

# 1. sample random heights (non-negative)
# using float64 for precision, requires_grad=False for plotting/evaluation only
height_values = torch.rand(P_val, dtype=torch.float64) 
# for more structured/sparse functions, you could try:
# height_values = torch.abs(torch.randn(P_val, dtype=torch.float64)) * (torch.rand(P_val, dtype=torch.float64) > 0.7).double()


# 2. compute integral of f(x)
integral_value = compute_integral_of_step_function(height_values, f_delta_x)

# 3. compute autoconvolution (f*f)(t) knot values
# (2*P_val + 1) values for t_m from 2*f_x_min to 2*f_x_max
autoconv_knot_vals = compute_autoconvolution_values(height_values, f_delta_x, P_val)
max_autoconv_value = torch.max(autoconv_knot_vals)

# 4. calculate the ratio for C1 estimate
# (max f*f(t)) / (integral f(x) dx)^2
if integral_value.item() == 0:
    c1_ratio = float('inf') if max_autoconv_value.item() > 0 else 0.0 # handle division by zero
else:
    c1_ratio = max_autoconv_value / (integral_value**2)

print(f"--- Function Details (P={P_val}) ---")
# print(f"Heights (h_i, first 10): {height_values.numpy()[:10]}") # uncomment if you want to see heights
print(f"Integral of f(x): {integral_value.item():.6f}")
print(f"Max value of autoconvolution (f*f)(t): {max_autoconv_value.item():.6f}")
print(f"Ratio max(f*f) / (integral(f))^2: {c1_ratio.item():.6f}")


# 5. plotting
# plot f(x)
plot_rendered_step_function(height_values.numpy(), f_interval, title=f"Random Step Function f(x) (P={P_val})")

# plot f*f(t)
# autoconvolution is defined on [2*f_x_min, 2*f_x_max]
conv_t_knots = np.linspace(2 * f_x_min, 2 * f_x_max, 2 * P_val + 1, dtype=float)
plot_rendered_convolution(conv_t_knots, autoconv_knot_vals.numpy(), title=f"Autoconvolution f*f(t) (P={P_val})")



## Plotting Google's solution

In [ ]:
# Also plot google's solution
h_params_google = return_height_params_google_simplex()
autoconv_knot_vals_google = compute_autoconvolution_values(h_params_google, f_delta_x, P_val)
max_autoconv_value_google = torch.max(autoconv_knot_vals_google)

plot_rendered_step_function(h_params_google.detach().numpy(), f_interval, title=f"Google's Step Function f(x) (P={P_val})")
plot_rendered_convolution(conv_t_knots, autoconv_knot_vals_google.detach().numpy(), title=f"Google'sAutoconvolution f*f(t) (P={P_val})")

## Algorithm 1: (Projected) Polyak subgradient method

The algorithm 
$$
h_+ = \mathrm{proj}_{\Delta_{2P}}\left(h - \frac{L(h) - L^*}{\| \nabla L(h) \|^2} \nabla L(h)\right)
$$
where $L^*$ is the target loss.

There is a subtlety here: we don't know what the optimal value is.

### Polyak with Random initialization

In [ ]:
P_val = 600
# Initialize random height parameters
h_params_polyak = torch.rand(P_val, dtype=torch.float64)
h_params_polyak.requires_grad = True
# Define the projection and loss
def projection(x): 
    return projection_simplex_pytorch(x, 2*P_val)
h_params_polyak.data = projection(h_params_polyak.data)
loss_fn = constrained_loss(P_val)


# Setting up history
if 'loss_history_polyak' not in locals():
    loss_history_polyak = []
if 'grad_norm_history_polyak' not in locals():
    grad_norm_history_polyak = []
if 'min_loss_found_polyak' not in locals():
    min_loss_found_polyak = float('inf')
if 'best_h_params_polyak' not in locals():
    best_h_params_polyak = h_params_polyak.data.clone()

# Setting up parameters for Polyak
max_iter = 100000
target_loss = 1.505
print_every = 1000

best_height_params_polyak, min_loss_found_polyak, loss_history_polyak, grad_norm_history_polyak = polyak_subgradient_method(h_params_polyak,loss_fn, max_iter, target_loss, print_every, loss_history_polyak, grad_norm_history_polyak, min_loss_found_polyak, best_h_params_polyak, projection)
print("Best loss found:", min_loss_found_polyak)
# semilogy plot of loss history
plt.semilogy(loss_history_polyak)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method: Loss History")
plt.show()


In [ ]:
autoconv_knot_vals_polyak = compute_autoconvolution_values(best_height_params_polyak, f_delta_x, P_val)
max_autoconv_value_polyak = torch.max(autoconv_knot_vals_polyak)

plot_rendered_step_function(best_height_params_polyak.detach().numpy(), f_interval, title=f"Step Function f(x) (After Polyak) (P={P_val})")
plot_rendered_convolution(conv_t_knots, autoconv_knot_vals_polyak.detach().numpy(), title=f"Autoconvolution f*f(t) (After Polyak) (P={P_val})")

torch.set_printoptions(precision=16, sci_mode=False)  # global setting
# need 16 digits of precision for the loss for the print statement to be correct

### Polyak started at the Google solution 

In [ ]:
P_val = 600
h_params_polyak_google = h_params_google.data.detach().clone() # initialize variable at google's solution.
h_params_polyak_google.requires_grad = True
# Define the projection and loss
def projection(x): 
    return projection_simplex_pytorch(x, 2*P_val)
h_params_polyak_google.data = projection(h_params_polyak_google.data)
loss_fn = constrained_loss(P_val)

# Setting up history
if 'loss_history_polyak_google' not in locals():
    loss_history_polyak_google = []
if 'grad_norm_history_polyak_google' not in locals():
    grad_norm_history_polyak_google = []
if 'min_loss_found_polyak_google' not in locals():
    min_loss_found_polyak_google = float('inf')
if 'best_h_params_polyak_google' not in locals():
    best_h_params_polyak_google = h_params_polyak_google.data.clone()
# if 'loss_history_polyak_google' not in locals():
# Setting up parameters for Polyak
max_iter = 10000
target_loss_google = 1.505292
print_every = 1000

best_height_params_polyak_google, min_loss_found_polyak_google, loss_history_polyak_google, grad_norm_history_polyak_google = polyak_subgradient_method(h_params_polyak_google, loss_fn, max_iter, target_loss_google, print_every, loss_history_polyak_google, grad_norm_history_polyak_google, min_loss_found_polyak_google, best_h_params_polyak_google, projection)

print("Best loss found (Google solution initialization):", min_loss_found_polyak_google)

# semilogy plot of loss history
plt.semilogy(loss_history_polyak_google)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method (Google solution initialization): Loss History")
plt.show()


In [ ]:
# Also plot google's solution
autoconv_knot_vals_google_polyak = compute_autoconvolution_values(best_height_params_polyak_google, f_delta_x, P_val)
max_autoconv_value_google_polyak = torch.max(autoconv_knot_vals_google_polyak)

plot_rendered_step_function(best_height_params_polyak_google.detach().numpy(), f_interval, title=f"Google's Step Function f(x) (After Polyak) (P={P_val})")
plot_rendered_convolution(conv_t_knots, autoconv_knot_vals_google_polyak.detach().numpy(), title=f"Google's Autoconvolution f*f(t) (After Polyak) (P={P_val})")

torch.set_printoptions(precision=16, sci_mode=False)  # global setting

print("best_height_params_polyak_google", best_height_params_polyak_google)
# need 16 digits of precision for the loss for the print statement to be correct

## Changing the loss function to remove constraints

We're going to precompose the loss function with a softmax. This implicitly constrains the functions to the simplex. I.e., we will optimize

$$
L(2P\mathrm{softmax}(w))
$$


In [ ]:
import torch.nn.functional as F

# Setting function parameters
P_val = 600  # number of pieces for the step function
f_interval = (-0.25, 0.25) # interval for f(x)
f_x_min, f_x_max = f_interval
f_delta_x = (f_x_max - f_x_min) / P_val
# Define optimization variable
height_params = torch.rand(P_val, dtype=torch.float64)
height_params.requires_grad = True

def loss_fn_softmax(log_h): 
    vals = 2*P_val*F.softmax(log_h, dim=0)
    max_f_conv_f_values_at_knots = compute_autoconvolution_values(vals, f_delta_x, P_val)
    return max_f_conv_f_values_at_knots.max()

# Compute loss and gradient at a random point.
loss = loss_fn_softmax(height_params)
print("loss", loss.item())
grad = torch.autograd.grad(loss, height_params)[0]


## Polyak on unconstrained (softmax) loss

In [ ]:

height_params_polyak = height_params.data.detach().clone() # initialize variable at value from test cell 
height_params_polyak.requires_grad = True

# if loss history exists, append to it
if 'loss_history_polyak' not in locals():
    loss_history_polyak = []

if 'grad_norm_history_polyak' not in locals():
    grad_norm_history_polyak = []
max_iter = 10000
target_loss = 1.505
print_every = 1000

if 'min_loss_found_polyak' not in locals():
    min_loss_found_polyak = float('inf')

if 'best_height_params_polyak' not in locals():
    best_height_params_polyak = height_params_polyak.data.clone()
else: 
    height_params_polyak.data = best_height_params_polyak.data.clone()

print("Starting Polyak subgradient method")
best_height_params_polyak, min_loss_found_polyak, loss_history_polyak, grad_norm_history_polyak = polyak_subgradient_method(height_params_polyak, loss_fn_softmax,max_iter=max_iter, target_loss=target_loss, print_every=print_every, loss_history=loss_history_polyak, grad_norm_history=grad_norm_history_polyak, min_loss_found   =min_loss_found_polyak, best_h_params=best_height_params_polyak)

print("Best loss found:", min_loss_found_polyak)

# semilogy plot of loss history
plt.semilogy(loss_history_polyak)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method: Loss History")
plt.show()

plt.semilogy(grad_norm_history_polyak)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method: Loss History")
plt.show()


In [31]:
def bfgs_method_softmax(h_params, loss_fn, max_iter=100000, lr=1.0, print_every=100, loss_history=[], grad_history=[], min_loss_found=float('inf'), best_h_params=None):
    optimizer = torch.optim.LBFGS([h_params], lr=lr)
    print("Starting BFGS method")
    for i in range(max_iter):
        # Compute loss and gradient
        def closure():
            h_params.grad = None
            loss = loss_fn(h_params)
            grad = torch.autograd.grad(loss, h_params)[0]
            h_params.grad = grad
            return loss
        optimizer.step(closure)

        grad_norm_squared = torch.norm(grad)
        loss_history.append(loss.item())        
        if i % print_every == 0:
            print(f"Iteration {i}: loss = {loss.item():.7f}, grad_norm = {torch.sqrt(grad_norm_squared).item():.7f}, min_loss_found = {min_loss_found:.6f}",)

        if loss.item() < min_loss_found:
            min_loss_found = loss.item()
            best_h_params = h_params.data.clone()
    return best_h_params, min_loss_found, loss_history, grad_history


In [ ]:

height_params_bfgs = height_params.data.detach().clone() # initialize variable at value from test cell 
height_params_bfgs.requires_grad = True

# if loss history exists, append to it
if 'loss_history_bfgs' not in locals():
    loss_history_bfgs = []

max_iter = 100000
target_loss = 1.52
print_every = 1
lr = 1.0

if 'min_loss_found_bfgs' not in locals():
    min_loss_found_bfgs = float('inf')

if 'best_height_params_bfgs' not in locals():
    best_height_params_bfgs = height_params_bfgs.data.clone()

best_height_params_bfgs, min_loss_found_bfgs, loss_history_bfgs = bfgs_method_softmax(height_params_bfgs, loss_fn_softmax, max_iter, lr=lr, print_every=print_every, loss_history=loss_history_bfgs, min_loss_found=min_loss_found_bfgs, best_h_params=best_height_params_bfgs)

print("Best loss found:", min_loss_found_bfgs)

# semilogy plot of loss history
plt.semilogy(loss_history_bfgs)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("BFGS method: Loss History")
plt.show()


In [ ]:

height_params_polyak = height_params.data.detach().clone() # initialize variable at value from test cell 
height_params_polyak.requires_grad = True

# if loss history exists, append to it
if 'loss_history_polyak' not in locals():
    loss_history_polyak = []

if 'grad_history_polyak' not in locals():
    grad_history_polyak = []
max_iter = 10000
target_loss = 1.505
print_every = 1000

if 'min_loss_found_polyak' not in locals():
    min_loss_found_polyak = float('inf')

if 'best_height_params_polyak' not in locals():
    best_height_params_polyak = height_params_polyak.data.clone()
else: 
    height_params_polyak.data = best_height_params_polyak.data.clone()

print("Starting Polyak subgradient method")
best_height_params_polyak, min_loss_found_polyak, loss_history_polyak, grad_history_polyak = polyak_subgradient_method_softmax(height_params_polyak, loss_fn_softmax, max_iter, target_loss, print_every, loss_history_polyak, grad_history_polyak, min_loss_found_polyak, best_height_params_polyak)

print("Best loss found:", min_loss_found_polyak)

# semilogy plot of loss history
plt.semilogy(loss_history_polyak)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method: Loss History")
plt.show()

plt.semilogy(grad_history_polyak)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Polyak subgradient method: Loss History")
plt.show()


In [64]:
def newton_method(h_params, loss_fn, max_iter=100000, target_loss=1.5053, print_every=100, loss_history_newton=[], min_loss_found_newton=float('inf'), best_height_params_newton=height_params.data.clone()):
    for i in range(max_iter):
        # Compute loss and gradient
        h_params.grad = None
        loss = loss_fn_softmax(h_params)
        grad = torch.autograd.grad(loss, h_params, create_graph=False)[0]
        hessian = torch.autograd.functional.hessian(loss_fn_softmax, h_params, create_graph=False)

        reg_hessian = torch.sqrt(10*torch.linalg.norm(grad))*torch.eye(torch.numel(h_params)) + hessian
        try:
            direction = torch.linalg.solve(reg_hessian, grad)
        except torch.linalg.LinAlgError:
            # if hessian is singular, use pseudo-inverse (least-squares solution)
            # this provides the minimum norm solution when the system is consistent and underdetermined
            # or the minimum norm least-squares solution if inconsistent
            direction = torch.linalg.lstsq(reg_hessian, grad).solution
        h_params.data -= direction

        # Project onto the simplex
        # with torch.no_grad():
            # h_params.data = projection_simplex_pytorch(h_params.data, 2*P_val)

        loss_history_newton.append(loss.item())        
        if i % print_every == 0:
            print(f"Iteration {i}: loss = {loss.item():.7f}, min_loss_found_newton  = {min_loss_found_newton:.6f}",)

        if loss.item() < min_loss_found_newton:
            min_loss_found_newton = loss.item()
            best_height_params_newton = h_params.data.clone()
    return best_height_params_newton, min_loss_found_newton, loss_history_newton


In [ ]:

height_params_newton = height_params.data.detach().clone() # initialize variable at value from test cell 
height_params_newton.requires_grad = True

# if loss history exists, append to it
if 'loss_history_newton' not in locals():
    loss_history_newton = []
max_iter = 100000
target_loss = 1.52
print_every = 1

if 'min_loss_found_newton' not in locals():
    min_loss_found_newton = float('inf')

if 'best_height_params_newton' not in locals():
    best_height_params_newton = height_params_newton.data.clone()

print("Starting Newton method")
best_height_params_newton, min_loss_found_newton, loss_history_newton = newton_method(height_params_newton, loss_fn, max_iter, target_loss, print_every, loss_history_newton, min_loss_found_newton, best_height_params_newton)

print("Best loss found:", min_loss_found_newton)

# semilogy plot of loss history
plt.semilogy(loss_history_newton)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Newton method: Loss History")
plt.show()
